In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.1,zoom_range=0.1,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Car Damage Dataset/level/training',
target_size = (256, 256),
batch_size = 10,
class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Car Damage Dataset/level/validation',
target_size = (256, 256),
batch_size = 10,
class_mode = 'categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
vgg = VGG16(input_shape=[256,256] + [3], weights='imagenet',include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
# our Layers - you can add more if you want
x = Flatten()(vgg.output)

In [ ]:
prediction = Dense(3,activation='softmax')(x)

In [ ]:
model1= Model(inputs=vgg.input, outputs=prediction)

In [ ]:
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [ ]:
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [ ]:
import sys
r=model1.fit_generator(
training_set,
validation_data=test_set,
epochs=25,
steps_per_epoch=979//10,
validation_steps=171//10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/25
97/97 [==============================] - 218s 2s/step - loss: 1.3064 - acc: 0.5624 - val_loss: 0.9411 - val_acc: 0.6529
Epoch 2/25
97/97 [==============================] - 17s 175ms/step - loss: 0.7407 - acc: 0.7245 - val_loss: 1.0167 - val_acc: 0.6118
Epoch 3/25
97/97 [==============================] - 17s 175ms/step - loss: 0.5716 - acc: 0.7988 - val_loss: 1.0148 - val_acc: 0.6588
Epoch 4/25
97/97 [==============================] - 18s 184ms/step - loss: 0.3917 - acc: 0.8545 - val_loss: 1.0578 - val_acc: 0.6647
Epoch 5/25
97/97 [==============================] - 17s 174ms/step - loss: 0.3238 - acc: 0.8844 - val_loss: 1.0943 - val_acc: 0.6882
Epoch 6/25
97/97 [==============================] - 17s 177ms/step - loss: 0.2443 - acc: 0.9051 - val_loss: 1.1098 - val_acc: 0.7118
Epoch 7/25
97/97 [==============================] - 18s 184ms/step - loss: 0.1743 - acc: 0.9443 - val_loss: 1.1019 - val_acc: 0.6647
Epoch 8/25
97/97 [==============================] - 17s 174ms/step - lo

In [ ]:
model1.save('level.h5')

In [ ]:
model=load_model('/content/drive/MyDrive/level.h5/')

In [ ]:
def detect(frame):
    img = cv2.resize(frame,(256,256)) #resizing the image to model trained image size
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) # uploaded image is in the form of BGR,so convert to RGB
    # scaling need to be done
    if(np.max(img)>1):
        img = img/255.0
    img = np.array([img]) #then to array fpormat
    prediction = model.predict (img)
    print(prediction)
    label = ["minor","moderate","severe"]
    preds = label[np.argmax(prediction)]
    return preds

In [ ]:
import numpy as np

In [ ]:
data = '/content/drive/MyDrive/Colab Notebooks/moderate-car-damage-300x263.jpg'
image = cv2.imread(data)
print (detect(image))

1/1 [==============================] - 0s 137ms/step
[[3.9014634e-05 9.9909949e-01 8.6151127e-04]]
moderate
